In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path       
from datetime import datetime 


In [3]:
base_dir = os.getcwd()
base_dir

'/home/sobottka/Documents/Competitions/Kaggle/optiver-trading-at-close/notebooks'

In [4]:
os.listdir()

['4.Optiver_Evaluation_v0 .ipynb',
 'structure.txt',
 '2.Optiver_Feature_Engineering_v0.ipynb',
 '1.Optiver_datapreprocessing_v0.ipynb',
 '3.Optiver_Model_Baseline_and_Advanced_v0.ipynb',
 '0.Optiver_EDA_v0.ipynb']

In [5]:
next_folder = os.chdir('..')

In [6]:
current_path = os.getcwd()
current_path

'/home/sobottka/Documents/Competitions/Kaggle/optiver-trading-at-close'

In [7]:
train_pd = pd.read_csv("train.csv")

In [8]:
train_pd.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [9]:
columns_sorted = sorted(train_pd.columns)
print(columns_sorted)
train_pd_sorted_by_columns = train_pd[columns_sorted]


['ask_price', 'ask_size', 'bid_price', 'bid_size', 'date_id', 'far_price', 'imbalance_buy_sell_flag', 'imbalance_size', 'matched_size', 'near_price', 'reference_price', 'row_id', 'seconds_in_bucket', 'stock_id', 'target', 'time_id', 'wap']


In [10]:
train_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
 16  row_id                   object 
dtypes: float64(11), int64(5), object(1)
memory usage: 679.4+ MB



### Time Handling

Sort by stock_id, time_id (or timestamp). This is critical for rolling features later.

Normalize time:

    Extract useful time features: second_in_day, minute_of_day.

    You’ll need these for intraday seasonality (prices behave differently at open/close).


In [11]:
#Sorting data by stock, date and seconds of the day 

train_sorted_pd = train_pd.sort_values(['stock_id', 'date_id','seconds_in_bucket'] ) #date_id represents different dates
train_sorted_pd.head(500)

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
191,0,0,10,1299772.70,1,1.000026,15261106.63,NaN,NaN,0.999812,13996.50,1.000026,23519.16,0.999892,0.389814,1,0_10_0
382,0,0,20,1299772.70,1,0.999919,15261106.63,NaN,NaN,0.999812,4665.50,0.999919,12131.60,0.999842,4.220009,2,0_20_0
573,0,0,30,1299772.70,1,1.000133,15261106.63,NaN,NaN,1.000026,55998.00,1.000133,46203.30,1.000085,5.450249,3,0_30_0
764,0,0,40,1218204.43,1,1.000455,15342674.90,NaN,NaN,1.000241,14655.95,1.000455,26610.45,1.000317,3.169775,4,0_40_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94820,0,9,0,4354435.11,1,0.999712,6004811.53,NaN,NaN,0.999916,5199.30,1.000120,7457.12,1.000000,-2.260208,495,9_0_0
95012,0,9,10,4189515.19,1,1.000018,6169731.45,NaN,NaN,0.999814,14419.23,1.000018,11969.42,0.999926,-3.859997,496,9_10_0
95204,0,9,20,4084343.18,1,0.999916,6274903.45,NaN,NaN,0.999814,5394.95,0.999916,9417.60,0.999851,-5.599856,497,9_20_0
95396,0,9,30,4020278.52,1,1.000018,6338968.12,NaN,NaN,0.999916,294.30,1.000018,25508.60,0.999917,-5.580187,498,9_30_0


In [12]:
# Does the same that the code above, but no so efficiently groupby(...).apply(lambda x: x.sort_values(...)) splits the DataFrame into groups, sorts each group separately, and then concatenates, which is slower and uses more memory.


#train_sorted_pd = train_pd.groupby('stock_id').apply(lambda x: x.sort_values('time_id')).reset_index(drop=True)


In [13]:
# asserting monotonicity (property of an ordered relationship that consistently preserver or reverses the order of elements it operates on)

assert train_sorted_pd.groupby('stock_id')['date_id'].is_monotonic_increasing.all()

In [14]:

# Set MultiIndex only if not already set
if not isinstance(train_sorted_pd.index, pd.MultiIndex) or \
   train_sorted_pd.index.names != ['stock_id', 'date_id', 'seconds_in_bucket']:
    train_sorted_pd = train_sorted_pd.set_index(['stock_id', 'date_id', 'seconds_in_bucket'])
train_sorted_pd

imbalance_size  imbalance_buy_sell_flag  \
stock_id date_id seconds_in_bucket                                            
0        0       0                      3180602.69                        1   
                 10                     1299772.70                        1   
                 20                     1299772.70                        1   
                 30                     1299772.70                        1   
                 40                     1218204.43                        1   
...                                            ...                      ...   
199      480     500                    2661783.46                       -1   
                 510                    2658917.61                       -1   
                 520                    1352558.88                       -1   
                 530                    1352558.88                       -1   
                 540                    1884285.71                       -1   

                                    reference_price  matched_size  far_price  \
stock_id date_id seconds_in_bucket                                             
0        0       0                         0.999812   13380276.64        NaN   
                 10                        1.000026   15261106.63        NaN   
                 20                        0.999919   15261106.63        NaN   
                 30                        1.000133   15261106.63        NaN   
                 40                        1.000455   15342674.90        NaN   
...                                             ...           ...        ...   
199      480     500                       1.002764   23969215.23   1.000859   
                 510                       1.002764   23972081.09   1.000859   
                 520                       1.001812   23978379.68   1.000859   
                 530                       1.001812   23978379.68   1.000859   
                 540                       1.002129   24073677.32   1.000859   

                                    near_price  bid_price   bid_size  \
stock_id date_id seconds_in_bucket                                     
0        0       0                         NaN   0.999812   60651.50   
                 10                        NaN   0.999812   13996.50   
                 20                        NaN   0.999812    4665.50   
                 30                        NaN   1.000026   55998.00   
                 40                        NaN   1.000241   14655.95   
...                                        ...        ...        ...   
199      480     500                  1.001494   1.002764  122246.18   
                 510                  1.001494   1.002764  677012.04   
                 520                  1.001494   1.001812  225361.65   
                 530                  1.001494   1.001812  285559.05   
                 540                  1.001494   1.002129  250081.44   

                                    ask_price   ask_size       wap    target  \
stock_id date_id seconds_in_bucket                                             
0        0       0                   1.000026    8493.03  1.000000 -3.029704   
                 10                  1.000026   23519.16  0.999892  0.389814   
                 20                  0.999919   12131.60  0.999842  4.220009   
                 30                  1.000133   46203.30  1.000085  5.450249   
                 40                  1.000455   26610.45  1.000317  3.169775   
...                                       ...        ...       ...       ...   
199      480     500                 1.003082  425296.17  1.002835 -7.209778   
                 510                 1.003082  347268.87  1.002974 -9.750128   
                 520                 1.002129  194630.52  1.001982  3.629923   
                 530                 1.002129  214513.32  1.001993  4.760027   
                 540                 1.002447  300167.56  1.002274 -6.

In [15]:
train_sorted_pd.loc[(slice(0,10),0,540)] #Multiindex first number stock, date, seconds in bucket

,,,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
stock_id,date_id,seconds_in_bucket,,,,,,,,,,,,,,
0,0,540,906957.07,1,0.999383,27183793.68,1.000241,0.999919,0.999276,111912.00,0.999383,9793.35,0.999375,9.759665,54,0_540_0
1,0,540,92236.28,-1,1.000425,3439290.16,0.999955,1.000425,1.000308,19235.99,1.001248,45494.13,1.000587,28.949976,54,0_540_1
2,0,540,0.00,0,0.999877,5121992.88,0.999877,0.999877,0.999772,18985.00,1.000667,380.04,1.000649,-19.270182,54,0_540_2
3,0,540,2446727.30,1,0.999440,42482476.86,0.999956,0.999870,0.999354,46468.00,0.999440,10456.20,0.999424,0.619888,54,0_540_3
4,0,540,0.00,0,1.001191,41712582.55,1.001191,1.001191,1.000914,70821.87,1.001260,4781.04,1.001238,-0.439882,54,0_540_4
5,0,540,0.00,0,1.001898,20081588.43,1.001898,1.001898,1.001613,124123.95,1.002387,51901.78,1.002159,-14.370084,54,0_540_5
6,0,540,1167106.04,1,1.000241,7429765.62,1.001564,1.000955,1.000031,4285.71,1.000367,201019.70,1.000038,6.469488,54,0_540_6
7,0,540,1528666.10,-1,0.998807,65698586.27,0.998511,0.998713,0.998698,64173.00,0.999118,75756.00,0.998891,-1.749992,54,0_540_7
8,0,540,0.00,0,0.999547,24625571.20,0.999547,0.999547,0.998515,4332.10,0.999690,357819.00,0.998529,22.460222,54,0_540_8


In [16]:
train_sorted_pd.iloc[26454]

imbalance_size               475513.69
imbalance_buy_sell_flag             -1
reference_price               0.999193
matched_size               41686415.27
far_price                     0.999017
near_price                    0.999017
bid_price                     0.999193
bid_size                     110123.01
ask_price                     0.999368
ask_size                     283817.38
wap                           0.999242
target                        0.760555
time_id                          26454
row_id                       480_540_0
Name: (0, 480, 540), dtype: object

In [17]:
# Number of rows per stock is 26454 (last row for each stock), for each day I have 54 rows
number_of_days_per_stock = 26454/54
number_of_days_per_stock

489.8888888888889

In [18]:
#Checking for duplicates

print(train_sorted_pd.duplicated())

stock_id  date_id  seconds_in_bucket
0         0        0                    False
                   10                   False
                   20                   False
                   30                   False
                   40                   False
                                        ...  
199       480      500                  False
                   510                  False
                   520                  False
                   530                  False
                   540                  False
Length: 5237980, dtype: bool


### Missing Values (Context-aware)

Prices (*_price): forward-fill within each (stock_id, date_id); if the very first row is missing, back-fill from the first available in that day, else drop that row (rare).

Sizes (*_size): forward-fill; if a day starts with NaNs, set to 0 only if your EDA showed that means “no displayed size” rather than missing.

Flags (imbalance_buy_sell_flag): treat 0 as “no imbalance,” not missing. If NaN exists, map NaN→0 after checking frequency.

Keep a mask column was_nan_any for later error analysis (missingness can be predictive in microstructure).




In [19]:
# Different methods to count the number of the rows

print(len(train_sorted_pd)) #number of rows
print(train_sorted_pd.shape[0]) # number of rows, shape [1] give the number of columns
print(train_sorted_pd.count() ) # counts non-NAN values per column
print(train_sorted_pd.count().sum() )#counts all non-NAN values in the whole DataFrame

5237980
5237980
imbalance_size             5237760
imbalance_buy_sell_flag    5237980
reference_price            5237760
matched_size               5237760
far_price                  2343638
near_price                 2380800
bid_price                  5237760
bid_size                   5237980
ask_price                  5237760
ask_size                   5237980
wap                        5237760
target                     5237892
time_id                    5237980
row_id                     5237980
dtype: int64
67578790


In [20]:
# First we drop the rows with missing values at the column target, since we can not use them for training
# but to have a better understand of each rows are being dropped we will store the rows dropped

mask = train_sorted_pd['target'].isna()
rows_to_drop_target = train_sorted_pd[mask]
rows_to_drop_target

imbalance_size  imbalance_buy_sell_flag  \
stock_id date_id seconds_in_bucket                                            
19       438     0                             NaN                       -1   
101      328     0                             NaN                        0   
131      35      0                             NaN                        0   
                 10                            NaN                        0   
                 20                            NaN                        0   
...                                            ...                      ...   
158      388     500                           NaN                        0   
                 510                           NaN                        0   
                 520                           NaN                        0   
                 530                           NaN                        0   
                 540                           NaN                        0   

                                    reference_price  matched_size  far_price  \
stock_id date_id seconds_in_bucket                                             
19       438     0                              NaN           NaN        NaN   
101      328     0                              NaN           NaN        NaN   
131      35      0                              NaN           NaN        NaN   
                 10                             NaN           NaN        NaN   
                 20                             NaN           NaN        NaN   
...                                             ...           ...        ...   
158      388     500                            NaN           NaN        NaN   
                 510                            NaN           NaN        NaN   
                 520                            NaN           NaN        NaN   
                 530                            NaN           NaN        NaN   
                 540                            NaN           NaN        NaN   

                                    near_price  bid_price  bid_size  \
stock_id date_id seconds_in_bucket                                    
19       438     0                         NaN        NaN       0.0   
101      328     0                         NaN        NaN       0.0   
131      35      0                         NaN        NaN       0.0   
                 10                        NaN        NaN       0.0   
                 20                        NaN        NaN       0.0   
...                                        ...        ...       ...   
158      388     500                       NaN        NaN       0.0   
                 510                       NaN        NaN       0.0   
                 520                       NaN        NaN       0.0   
                 530                       NaN        NaN       0.0   
                 540                       NaN        NaN       0.0   

                                    ask_price  ask_size  wap  target  time_id  \
stock_id date_id seconds_in_bucket                                              
19       438     0                        NaN       0.0  NaN     NaN    24090   
101      328     0                        NaN       0.0  NaN     NaN    18040   
131      35      0                        NaN       0.0  NaN     NaN     1925   
                 10                       NaN       0.0  NaN     NaN     1926   
                 20                       NaN       0.0  NaN     NaN     1927   
...                                       ...       ...  ...     ...      ...   
158      388     500                      NaN       0.0  NaN     NaN    21390   
                 510                      NaN       0.0  NaN     NaN    21391   
                 520                      NaN       0.0  NaN     NaN    21392   
                 530                      NaN       0.0  NaN     NaN    21393   
                 540                      NaN       0.0  NaN     NaN   

In [21]:
# Now we can drop the rows at the dataframe of the column target
train_sorted_pd = train_sorted_pd.dropna(subset=['target'])
print(train_sorted_pd.isna().sum())
train_sorted_pd.head()

imbalance_size                 132
imbalance_buy_sell_flag          0
reference_price                132
matched_size                   132
far_price                  2894254
near_price                 2857092
bid_price                      132
bid_size                         0
ask_price                      132
ask_size                         0
wap                            132
target                           0
time_id                          0
row_id                           0
dtype: int64


imbalance_size  imbalance_buy_sell_flag  \
stock_id date_id seconds_in_bucket                                            
0        0       0                      3180602.69                        1   
                 10                     1299772.70                        1   
                 20                     1299772.70                        1   
                 30                     1299772.70                        1   
                 40                     1218204.43                        1   

                                    reference_price  matched_size  far_price  \
stock_id date_id seconds_in_bucket                                             
0        0       0                         0.999812   13380276.64        NaN   
                 10                        1.000026   15261106.63        NaN   
                 20                        0.999919   15261106.63        NaN   
                 30                        1.000133   15261106.63        NaN   
                 40                        1.000455   15342674.90        NaN   

                                    near_price  bid_price  bid_size  \
stock_id date_id seconds_in_bucket                                    
0        0       0                         NaN   0.999812  60651.50   
                 10                        NaN   0.999812  13996.50   
                 20                        NaN   0.999812   4665.50   
                 30                        NaN   1.000026  55998.00   
                 40                        NaN   1.000241  14655.95   

                                    ask_price  ask_size       wap    target  \
stock_id date_id seconds_in_bucket                                            
0        0       0                   1.000026   8493.03  1.000000 -3.029704   
                 10                  1.000026  23519.16  0.999892  0.389814   
                 20                  0.999919  12131.60  0.999842  4.220009   
                 30                  1.000133  46203.30  1.000085  5.450249   
                 40                  1.000455  26610.45  1.000317  3.169775   

                                    time_id  row_id  
stock_id date_id seconds_in_bucket                   
0        0       0                        0   0_0_0  
                 10                       1  0_10_0  
                 20                       2  0_20_0  
                 30                       3  0_30_0  
                 40                       4  0_40_0

In [22]:
print(f'{(train_sorted_pd.isna().sum() / len(train_sorted_pd))*100} \n ----------------------- \n')
cols_with_na = train_sorted_pd.columns[train_sorted_pd.isna().sum() > 0]


print(f'Columns with na:{cols_with_na} \n ------------------------\n')
train_with_na = train_sorted_pd[cols_with_na]
train_with_na

imbalance_size              0.002520
imbalance_buy_sell_flag     0.000000
reference_price             0.002520
matched_size                0.002520
far_price                  55.256084
near_price                 54.546600
bid_price                   0.002520
bid_size                    0.000000
ask_price                   0.002520
ask_size                    0.000000
wap                         0.002520
target                      0.000000
time_id                     0.000000
row_id                      0.000000
dtype: float64 
 ----------------------- 

Columns with na:Index(['imbalance_size', 'reference_price', 'matched_size', 'far_price',
       'near_price', 'bid_price', 'ask_price', 'wap'],
      dtype='object') 
 ------------------------



imbalance_size  reference_price  \
stock_id date_id seconds_in_bucket                                    
0        0       0                      3180602.69         0.999812   
                 10                     1299772.70         1.000026   
                 20                     1299772.70         0.999919   
                 30                     1299772.70         1.000133   
                 40                     1218204.43         1.000455   
...                                            ...              ...   
199      480     500                    2661783.46         1.002764   
                 510                    2658917.61         1.002764   
                 520                    1352558.88         1.001812   
                 530                    1352558.88         1.001812   
                 540                    1884285.71         1.002129   

                                    matched_size  far_price  near_price  \
stock_id date_id seconds_in_bucket                                        
0        0       0                   13380276.64        NaN         NaN   
                 10                  15261106.63        NaN         NaN   
                 20                  15261106.63        NaN         NaN   
                 30                  15261106.63        NaN         NaN   
                 40                  15342674.90        NaN         NaN   
...                                          ...        ...         ...   
199      480     500                 23969215.23   1.000859    1.001494   
                 510                 23972081.09   1.000859    1.001494   
                 520                 23978379.68   1.000859    1.001494   
                 530                 23978379.68   1.000859    1.001494   
                 540                 24073677.32   1.000859    1.001494   

                                    bid_price  ask_price       wap  
stock_id date_id seconds_in_bucket                                  
0        0       0                   0.999812   1.000026  1.000000  
                 10                  0.999812   1.000026  0.999892  
                 20                  0.999812   0.999919  0.999842  
                 30                  1.000026   1.000133  1.000085  
                 40                  1.000241   1.000455  1.000317  
...                                       ...        ...       ...  
199      480     500                 1.002764   1.003082  1.002835  
                 510                 1.002764   1.003082  1.002974  
                 520                 1.001812   1.002129  1.001982  
                 530                 1.001812   1.002129  1.001993  
                 540                 1.002129   1.002447  1.002274  

[5237892 rows x 8 columns]

### Analyzing missing data:

For the columns that present **0,25%** of missing values, it is reasonable to think to use a **forward fill (carry last available quote, since markets are continuous)** approach. 

However, for **Far/Near price**:

far_price - The crossing price that will maximize the number of shares matched based on auction interest only. This calculation excludes continuous market orders.

near_price - The crossing price that will maximize the number of shares matched based auction and continuous market orders.

We have roughly 54% to 55%, which arises the question should we include these features in our model? My approach will be using trial and error, I will train my model with the column and without the column, and check how my model performs.

Treat missing as “not available” information **(i.e. add a binary flag column like far_price_missing)**. This preserves the information that sometimes it’s just not provided.

In [23]:
# So now, I will create two columns for missing values on far_price and near_price, they will tell me if I have the information of if I do not

train_sorted_pd['far_missing'] = train_sorted_pd['far_price'].isna().astype(int)
train_sorted_pd['near_missing'] = train_sorted_pd['near_price'].isna().astype(int)

/tmp/ipykernel_1598690/512769262.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd['far_missing'] = train_sorted_pd['far_price'].isna().astype(int)
/tmp/ipykernel_1598690/512769262.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd['near_missing'] = train_sorted_pd['near_price'].isna().astype(int)


In [24]:
#Checking my new dataframe

train_sorted_pd[train_sorted_pd['far_missing']<1].head()

imbalance_size  imbalance_buy_sell_flag  \
stock_id date_id seconds_in_bucket                                            
0        0       300                           0.0                        0   
                 310                           0.0                        0   
                 320                           0.0                        0   
                 330                           0.0                        0   
                 340                           0.0                        0   

                                    reference_price  matched_size  far_price  \
stock_id date_id seconds_in_bucket                                             
0        0       300                       1.000241   26670118.88   1.000241   
                 310                       0.999919   26670118.88   0.999919   
                 320                       0.999919   26670118.88   0.999919   
                 330                       0.999812   26670118.88   0.999812   
                 340                       0.999491   26670118.88   0.999491   

                                    near_price  bid_price  bid_size  \
stock_id date_id seconds_in_bucket                                    
0        0       300                  1.000241   1.000026  19319.31   
                 310                  0.999919   0.999812  22767.64   
                 320                  0.999919   0.999705  49262.40   
                 330                  0.999812   0.999705  27990.00   
                 340                  0.999491   0.999169  18743.25   

                                    ask_price  ask_size       wap     target  \
stock_id date_id seconds_in_bucket                                             
0        0       300                 1.000241  16149.55  1.000143 -10.039806   
                 310                 0.999919  18664.00  0.999871  -6.039739   
                 320                 0.999919  37607.96  0.999826  -5.440116   
                 330                 0.999812  14276.43  0.999776  -7.439852   
                 340                 0.999383  40012.83  0.999237  -2.700090   

                                    time_id   row_id  far_missing  \
stock_id date_id seconds_in_bucket                                  
0        0       300                     30  0_300_0            0   
                 310                     31  0_310_0            0   
                 320                     32  0_320_0            0   
                 330                     33  0_330_0            0   
                 340                     34  0_340_0            0   

                                    near_missing  
stock_id date_id seconds_in_bucket                
0        0       300                           0  
                 310                           0  
                 320                           0  
                 330                           0  
                 340                           0

In [25]:
#Now I need to forward fill the columns which contains 0.25% of missing values

# 1. Forward fill the small-percentage missing values
small_missing_cols = ['imbalance_size','reference_price','matched_size',
                      'bid_price','ask_price','wap']
train_sorted_pd[small_missing_cols] = train_sorted_pd[small_missing_cols].fillna(method='ffill')
train_sorted_pd.isna().sum()

/tmp/ipykernel_1598690/3250190379.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_sorted_pd[small_missing_cols] = train_sorted_pd[small_missing_cols].fillna(method='ffill')
/tmp/ipykernel_1598690/3250190379.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[small_missing_cols] = train_sorted_pd[small_missing_cols].fillna(method='ffill')


imbalance_size                   0
imbalance_buy_sell_flag          0
reference_price                  0
matched_size                     0
far_price                  2894254
near_price                 2857092
bid_price                        0
bid_size                         0
ask_price                        0
ask_size                         0
wap                              0
target                           0
time_id                          0
row_id                           0
far_missing                      0
near_missing                     0
dtype: int64

### Handling Outliers

**Identify Which Columns Need Outlier Handling**

For Optiver TAC dataset, focus on numeric columns that affect price movement predictions:

Prices: bid_price, ask_price, reference_price, wap, far_price, near_price

Sizes / volumes: bid_size, ask_size, matched_size, imbalance_size

Derived features (later in feature engineering): spreads, ratios, etc.


**Decide How to Detect Outliers**

**Statistical Methods**

    1. Percentile clipping (Winsorizing)
        lower = df[col].quantile(0.005)
        upper = df[col].quantile(0.995)
        df[col] = df[col].clip(lower, upper)


    2. Z-score Method
        Calculate z = (x - mean) / std within a stock or day.

        Cap |z| > 3 or 4.

        Useful if distributions differ across stocks.

    3. IQR (Interquantile Range)
        Q1 = df[col].quantile(0.25)

        Q3 = df[col].quantile(0.75)

        Outliers: x < Q1 - 1.5*IQR or x > Q3 + 1.5*IQR

        Sometimes better for skewed distributions.


**Contextual / Market-Specific Methods**

    Bid / Ask Price Constraints

        Ask ≥ Bid always.

        Reference price usually close to WAP; if WAP deviates >5-10% from reference, treat as outlier.

    Imbalance / Matched Size

        If imbalance_size is negative when flag = 1, it’s likely an error.

        Cap extreme volumes to avoid skewing features like imbalance ratios.

    Time-Series Smoothness

        Sudden jump in wap or bid_price > X% in one second might be a misrecord; smooth or clip.

**Implementation Steps**
    1. Group by stock_id and optionally date_id

        Outliers are relative to each stock; one stock’s price might be 1000x another’s.
    
    2. Compute per-stock statistics

        group = df.groupby('stock_id')
        df['wap_clipped'] = group['wap'].transform(lambda x: x.clip(x.quantile(0.005), x.quantile(0.995)))

    3. Decide on treatment

        Clip / Winsorize → preserves row, reduces influence.

        Set to NaN / interpolate → can be done if value is obviously wrong.

        Drop row → rarely, if the row is corrupted across many columns.

**Practical Tips for Quant Modeling**

    Always clip based on training data stats, not the test set, to avoid leakage.

    Create indicator columns for extreme values → sometimes “being an outlier” is predictive.

    Visualize distributions after clipping:

        import seaborn as sns
        sns.boxplot(x='stock_id', y='wap', data=df.sample(5000))


| Step                                                             | Method                         | Notes                                         |
| ---------------------------------------------------------------- | ------------------------------ | --------------------------------------------- |
| Prices (`wap`, `bid_price`, `ask_price`)                         | Percentile clipping 0.5%-99.5% | Preserve order book structure                 |
| Sizes (`bid_size`, `ask_size`, `matched_size`, `imbalance_size`) | Winsorize or cap               | Avoid extreme ratios dominating               |
| Far / Near prices                                                | Missing + clip                 | Only if used in features                      |
| Indicator for clipped values                                     | Binary flag                    | Could help ML model recognize unusual events  |
| Temporal smoothing                                               | Optional                       | If single-second jump is extreme, interpolate |


**Jane Street / Optiver mindset:**

   - You never just “remove outliers” blindly.

   - Ask: Is this extreme value a market reality (volatility) or an error in data?

   - Preserve real market extremes — they are informative signals. Only remove obvious errors.

In [26]:
price_columns = ['wap', 'bid_price', 'ask_price']

for col in price_columns: 
    lower = train_sorted_pd[col].quantile(0.005)
    upper = train_sorted_pd[col].quantile(0.995)
    train_sorted_pd[col] = train_sorted_pd[col].clip(lower, upper)

/tmp/ipykernel_1598690/3082504089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col] = train_sorted_pd[col].clip(lower, upper)
/tmp/ipykernel_1598690/3082504089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col] = train_sorted_pd[col].clip(lower, upper)
/tmp/ipykernel_1598690/3082504089.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

**Winsorizing: statistical term, usually symmetric, replaces with boundary values.**

**Percentile Clipping: coding term, you specify quantile thresholds (not always symmetric).**

1. What is Winsorizing?

Winsorizing means replacing extreme values with the nearest acceptable percentile boundary, instead of dropping them.

Example (1%–99% winsorization):

If a value < 1st percentile → replace it with the 1st percentile.

If a value > 99th percentile → replace it with the 99th percentile.

So the distribution is “squished” at the tails, but no rows are removed.

🔹 2. What is Capping / Clipping?

Capping (a.k.a. clipping) is essentially the same as winsorizing in implementation:

You set a hard min and max (usually based on quantiles).

Any values outside are set equal to the boundaries.

The difference is mainly terminology:

In statistics, “winsorizing” usually refers to symmetric tail treatment (like 5%–95%).

In coding (pandas, numpy), you’ll usually see clip(), which is equivalent.

🔹 3. Why Use Winsorization/Capping for Size Columns?

Size features:

bid_size, ask_size, matched_size, imbalance_size
tend to have heavy-tailed distributions — most orders are small, but occasionally there are massive block trades.

If you don’t cap, a single huge bid_size could dominate your model (especially linear models).

But you don’t want to delete these rows, because the fact that a large order appeared might be informative.

Winsorizing preserves the row, just prevents extreme distortion.

In [27]:
size_cols = ['bid_size', 'ask_size', 'matched_size', 'imbalance_size']

for col in size_cols:
    lower = train_sorted_pd[col].quantile(0.01)   # 1st percentile
    upper = train_sorted_pd[col].quantile(0.99)   # 99th percentile
    
    train_sorted_pd[col + '_winsor'] = train_sorted_pd[col].clip(lower, upper)
    
    # Optional: indicator for whether it was capped
    train_sorted_pd[col + '_is_capped'] = (
        (train_sorted_pd[col] < lower) | (train_sorted_pd[col] > upper)
    ).astype(int)


/tmp/ipykernel_1598690/971069179.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col + '_winsor'] = train_sorted_pd[col].clip(lower, upper)
/tmp/ipykernel_1598690/971069179.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_sorted_pd[col + '_is_capped'] = (
/tmp/ipykernel_1598690/971069179.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 


### Memory Optimization

Datasets are huge in Optiver, so:

    Downcast columns (float64 → float32, int64 → int32).

    Use category dtype for stock_id.

    Save intermediate results as feather/parquet instead of CSV.


In [28]:
def reduce_memory_usage(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
            else:
                df[col] = df[col].astype(np.float32)
        else:
            df[col] = df[col].astype('category')
    return df





### Split Train/Valid/Test

Since this is time series:

    Split by time_id (or date), not randomly.

    Example: Train on first 80% of time blocks, validate on next 10%, test on last 10%.

    (Later, use rolling CV for more robustness).

Train → model learns.

Validation → tune hyperparameters.

Test → final unbiased evaluation.

How (time-series case):

Do not shuffle!

Split by time (earlier dates = train, middle = valid, latest = test).

1. Purpose of Each Split

Train set → Used to fit the model parameters (e.g., regression weights, neural net weights).

Validation set → Used to tune hyperparameters (things you don’t directly fit, like learning rate, number of layers, window size, regularization strength). You look at performance on validation data to decide which model configuration is “best.”

Test set → Used once at the very end for an unbiased performance estimate. It mimics future unseen data.

2. Why We Need Validation

If you only had train/test:

You’d try different hyperparameters, but each time you’d be peeking at the test set.

This makes the test set “contaminated” because you’re indirectly fitting to it.

So the validation set is a sandbox where you’re allowed to try things. The test is your final exam where you can’t look ahead.

3. Time Series Specifics

In time series, you cannot shuffle because future must never leak into the past.

Train → earlier dates (model learns patterns).

Validation → the “next chunk” of time. You try hyperparameters and see how the model forecasts.

Test → the most recent chunk. Used only at the end for unbiased evaluation.

In [ ]:
# Need to be checked

train = df[df['date'] < '2017-01-01']
valid = df[(df['date'] >= '2017-01-01') & (df['date'] < '2018-01-01')]
test  = df[df['date'] >= '2018-01-01']



### Pipeline Setup

Organize preprocessing so you can reproduce it later:

    Write functions: clean_missing(df), normalize(df), align_data(ob, tr).

    Save processed datasets for reuse.
